In [1]:
!pip install -r requirements.txt

  Cloning https://github.com/TinArmEngineering/tinarm.git (to revision 0.1.49) to /tmp/pip-req-build-bmj5byv0
  Running command git clone --filter=blob:none --quiet https://github.com/TinArmEngineering/tinarm.git /tmp/pip-req-build-bmj5byv0
  Running command git checkout -q a2fdc7ce19efad177874d9b06fdab31b85f00681
  Resolved https://github.com/TinArmEngineering/tinarm.git to commit a2fdc7ce19efad177874d9b06fdab31b85f00681
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import matplotlib.pyplot as plt
import pint
from tinarm import NameQuantityPair
from tinarm import Quantity 
from tinarm.api import JOB_STATUS

import tinarm
import logging
import time
import requests
import uuid
import random
import yaml
import pandas as pd
import pint_pandas


In [4]:

LOGGING_LEVEL = logging.INFO
STATUS_JOB = {value:key for key,value in JOB_STATUS.items()}

In [5]:

### Configure Logging
logger = logging.getLogger()
logger.setLevel(LOGGING_LEVEL)
logger.info(f"tinarm version {tinarm.__version__}")
logger.info(f"pint_pandas version {pint_pandas.__version__}")

q = pint_pandas.PintType.ureg
q.setup_matplotlib()

2024-02-01 14:37:29,078 - NoJobId - INFO - Kebnekaise - 1683099798.py-><module>() - tinarm version 0.1
2024-02-01 14:37:29,079 - NoJobId - INFO - Kebnekaise - 1683099798.py-><module>() - pint_pandas version 0.5


Log in to Tin Arm Engineering's machine solver, and under profile, retrieve your API key. 
Do not commit your API key to a repository, and consider it like a password.  A good way to keep it out of this code base is to use a configuation file, or environment variable. We will want other things cofigured too, so let's use a yaml file for convenience. 

create a `configurations.yaml` file in this directory with the content.
```yaml
api_key: 668952058c490d0a296da0abb966****
root_url: http://server-go:4300
```
But of course replace the number with your api key

You can then add that file to your `.gitignore` file by executing ```echo 'configurations.yaml' >> .gitignore```

```
!echo 'configurations.yaml' >> .gitignore
```

In [6]:
with open("configurations.yaml", "r") as f:
    config = yaml.safe_load(f)

# Stator

In [7]:
stator_parameters = {
    "slot_liner_thikness": 300 * q.um,
    "stator_bore": 8.20 * q.cm,
    "tooth_tip_depth": 1.5 * q.mm,
    "slot_opening": 1.5 * q.mm,
    "tooth_width": 9.8 * q.mm,
    "stator_outer_diameter": 0.136 * q.m,
    "back_iron_thickness": 5.5 * q.mm,
    "stator_internal_radius": 500 * q.um,
    "number_slots": 12 * q.count,
    "tooth_tip_angle": 70 * q.degrees
    }

air_gap_length = 1 * q.mm

## Rotor
Surface mounted Breadloaf magnets

In [8]:
rotor_parameters = {
    "rotor_od": stator_parameters["stator_bore"] - 2 * air_gap_length,
    "rotor_bore": 40 * q.mm,
    "banding_thickness": 0.5 * q.mm,
    "number_poles": 10 * q.count,
    "magnet_thickness": 4.5 * q.millimeter,
    "magnet_pole_arc": 150 * q.degrees,
    "magnet_inset": 0.25 * q.millimeter
    }

## Simulation Parameters

In [9]:
simulation_parameters = {
       "samples_per_electrical_period": 180 * q.count/q.turn,
        "timestep_intervals": 180 * q.count,
        "active_length": 65 * q.mm}


In [10]:
winding_parameters = {
    "symmetry": 2 * q.count,
    "number_phases": 3 * q.count,
    "number_layers": 2 * q.count,
    "coil_span": 1 * q.count,
    "turns_per_coil": 43 * q.count,
    "empty_slots": 0 * q.count,
    "fill_factor": 42 * q.percent
    }

## Operating Point

In [11]:
import numpy as np

current_density_grid, angle_grid = np.meshgrid(np.linspace(2, 10, 5) * q.A/q.mm**2, np.linspace(250, 250+45, 5) * q.degrees)
current_density_grid.reshape(-1), angle_grid.reshape(-1)

(array([ 2.,  4.,  6.,  8., 10.,  2.,  4.,  6.,  8., 10.,  2.,  4.,  6.,
         8., 10.,  2.,  4.,  6.,  8., 10.,  2.,  4.,  6.,  8., 10.]) <Unit('ampere / millimeter ** 2')>,
 array([250.  , 250.  , 250.  , 250.  , 250.  , 261.25, 261.25, 261.25,
        261.25, 261.25, 272.5 , 272.5 , 272.5 , 272.5 , 272.5 , 283.75,
        283.75, 283.75, 283.75, 283.75, 295.  , 295.  , 295.  , 295.  ,
        295.  ]) <Unit('degree')>)

In [12]:
operating_points = [{
    'current_density': cd,
    'current_angle': angle,
    "simulated_speed": 2400 * q.rpm,
    } for cd, angle in zip(current_density_grid.reshape(-1), angle_grid.reshape(-1))]



In [13]:
class Machine(object):
    def __init__(self, stator, rotor, winding):

        self.stator = stator
        self.rotor = rotor
        self.winding = winding

    def __repr__(self) -> str:
        return f"Machine({self.stator}, {self.rotor}, {self.winding})"
    
    def to_api(self):
        stator_api = [NameQuantityPair("stator",
                                        k,
                                        Quantity(*self.stator[k].to_tuple())) for k in  self.stator]
        rotor_api = [NameQuantityPair("rotor",
                                        k,
                                        Quantity(*self.rotor[k].to_tuple())) for k in  self.rotor]
        winding_api = [NameQuantityPair("winding",
                                        k,
                                        Quantity(*self.winding[k].to_tuple())) for k in  self.winding]
        data = []
        data.extend(list(x.to_dict() for x in stator_api))
        data.extend(list(x.to_dict() for x in rotor_api))
        data.extend(list(x.to_dict() for x in winding_api))
        return data
        
    
class Job(object):
    def __init__(self, machine: Machine, operating_point, simulation, title=None):
        if title is None:
            self.title = self.generate_title()
        else:
            self.title = title
        self.type = "electromagnetic_spmbrl_fscwseg"
        self.status = 0
        self.machine = machine
        self.operating_point = operating_point
        self.simulation = simulation
        
    def __repr__(self) -> str:
        return f"Job({self.machine}, {self.operating_point}, {self.simulation})"
    
    def generate_title(self):
        "gets a random title from the wordlists"
        random_offset = random.randint(500, 286797)
        response = requests.get("https://github.com/taikuukaits/SimpleWordlists/raw/master/Wordlist-Adjectives-All.txt",
                                headers={'Range': 'bytes={1}-{0}'.format(random_offset, random_offset-500), 'accept-encoding': 'identity'})
        adjective = random.choice(response.text.splitlines()[1:-1])
        random_offset = random.randint(500, 871742)
        response = requests.get("https://github.com/taikuukaits/SimpleWordlists/raw/master/Wordlist-Nouns-All.txt",
                                headers={'Range': 'bytes={1}-{0}'.format(random_offset, random_offset-500), 'accept-encoding': 'identity'})
        noun = random.choice(response.text.splitlines()[1:-1])
        title = f"{adjective}-{noun}"
        return title 

    def to_api(self):
        job =  {"status": 0,
                "title": self.title,
                "type": self.type,
                "tasks": 11,
                "data": []}
        
        operating_point_api = [NameQuantityPair("operating_point",
                                        k,
                                        Quantity(*self.operating_point[k].to_tuple())) for k in  self.operating_point]
        
        simulation_api = [NameQuantityPair("simulation",
                                        k,
                                        Quantity(*self.simulation[k].to_tuple())) for k in  self.simulation]
        job["data"].extend(list(x.to_dict() for x in operating_point_api))
        job["data"].extend(list(x.to_dict() for x in simulation_api))
        job["data"].extend(self.machine.to_api())
        return job
    
    def run(self):
        pass

In [14]:
class Api:
    """
    The TAE User API
    """

    def __init__(self, root_url, api_key, org_id=None):
        """
        Initialize the API
        """
        self._root_url = root_url
        self._api_key = api_key
        self._org_id = org_id
        self._node_id = None

        logger.info(f"root_url: {self._root_url}")

    def get_job(self, job_id):
        """
        Get a job from the TAE API
        """
        response = requests.get(
            url=f"{self._root_url}/jobs/{job_id}?apikey={self._api_key}",
        )
        response.raise_for_status()
        return response.json()
    
    def create_job(self, job):
        """
        Create a job for the TAE API
        """
        response = requests.post(
            url=f"{self._root_url}/jobs/?apikey={self._api_key}&org_id={self._org_id}",
            json=job.to_api()
        )
        response.raise_for_status()
        if response.status_code == 200:
            job.id = response.json()["id"]
        return response.json()
    
    def update_job_status(self, job_id, status):
        """
        Update a job status
        """
        url = f"{self._root_url}/jobs/{job_id}/status/{status}?node_id={self._node_id}&apikey={self._api_key}"
        logger.info(f"Updating job status: {url}")

        response = requests.put(url=url)
        return response.json()
    
    

In [15]:
api = Api(config["root_url"], config["api_key"], config["org_id"])

2024-02-01 14:37:39,082 - NoJobId - INFO - Kebnekaise - 3503141280.py->__init__() - root_url: https://api.build.tinarmengineering.com


In [16]:
print('jerking ({current_density:2.4~P}, {current_angle:2.4~P})'.format(**operating_points[0]))

jerking (2.0 A/mm², 250.0 deg)


In [18]:
m1 = Machine(stator_parameters, rotor_parameters, winding_parameters)

auto_title = Job(m1, operating_points[0], simulation_parameters).generate_title()

In [24]:
'{title}-{current_density:2.4~P}-{current_angle:2.4~P}'.format(title=auto_title, **operating_points[0]).replace(" ", "-")

'santalaceous-conformist-2.0-A/mm²-250.0-deg'

In [25]:




jobs = [Job(m1, op, simulation_parameters, title='{title}-{current_density:2.4~P}-{current_angle:2.4~P}'.format(title=auto_title, **op).replace(" ", "-")) for op in operating_points]



jobs_mapping = map(lambda x: api.create_job(x), jobs)

In [26]:
#it is this line that actually creates the jobs
jobs_result = list(jobs_mapping)


In [31]:
[STATUS_JOB[api.get_job(j['id'])['status']] for j in jobs_result if j['id'] !="65bb9fd93b76d9083bf872e6"]

['Complete',
 'Complete',
 'Complete',
 'Complete',
 'Complete',
 'PostProcess',
 'Complete',
 'Complete',
 'Complete',
 'Complete',
 'Complete',
 'Complete',
 'Complete',
 'Complete',
 'Complete',
 'Complete',
 'Complete',
 'Complete',
 'Complete',
 'Complete',
 'Complete',
 'Complete',
 'Complete',
 'Complete']

In [28]:
for j in jobs_result:
    res = api.update_job_status(j['id'], JOB_STATUS['QueuedForMeshing'])
    time.sleep(0.05)
    print(res)

2024-02-01 14:46:45,763 - NoJobId - INFO - Kebnekaise - 3503141280.py->update_job_status() - Updating job status: https://api.build.tinarmengineering.com/jobs/65bb9fd93b76d9083bf872e6/status/10?node_id=None&apikey=668952058c490d0a296da0abb9660534
mongo: no documents in result
2024-02-01 14:46:46,080 - NoJobId - INFO - Kebnekaise - 3503141280.py->update_job_status() - Updating job status: https://api.build.tinarmengineering.com/jobs/65bb9fda3b76d9083bf872e7/status/10?node_id=None&apikey=668952058c490d0a296da0abb9660534
{'id': '65bb9fda3b76d9083bf872e7', 'owner_org_id': '65a9291658b95253163001ab', 'owner_user_id': '659bf5a7b7135efed48311bd', 'creation_date': '2024-02-01T13:42:50Z', 'status': 10, 'title': 'santalaceous-conformist-4.0-A/mm²-250.0-deg', 'type': 'electromagnetic_spmbrl_fscwseg', 'node_id': 'None', 'data': [{'section': 'operating_point', 'name': 'current_density', 'value': {'magnitude': 4, 'units': [{'name': 'ampere', 'exponent': 1}, {'name': 'millimeter', 'exponent': -2}]}},

example of hashing the input

In [57]:
jobs_result[0]['data'][0]['value']

{'magnitude': 2,
 'units': [{'name': 'ampere', 'exponent': 1},
  {'name': 'millimeter', 'exponent': -2}]}

In [58]:
def decode(enc):
    return q.Quantity.from_tuple(
        (enc["magnitude"], ((e["name"], e["exponent"]) for e in enc["units"]))
    )

In [64]:
stator_params = {x['name']:decode(x['value']) for x in jobs_result[0]['data'] if x['section']=='stator'}
stator_params

{'slot_liner_thikness': 300 <Unit('micrometer')>,
 'stator_bore': 8.2 <Unit('centimeter')>,
 'tooth_tip_depth': 1.5 <Unit('millimeter')>,
 'slot_opening': 1.5 <Unit('millimeter')>,
 'tooth_width': 9.8 <Unit('millimeter')>,
 'stator_outer_diameter': 0.136 <Unit('meter')>,
 'back_iron_thickness': 5.5 <Unit('millimeter')>,
 'stator_internal_radius': 500 <Unit('micrometer')>,
 'number_slots': 12 <Unit('count')>,
 'tooth_tip_angle': 70 <Unit('degree')>}

In [90]:
base_unit_stator_params = {k:v.to_base_units() for k,v in stator_params.items()}
base_unit_stator_params


{'slot_liner_thikness': 0.0003 <Unit('meter')>,
 'stator_bore': 0.08199999999999999 <Unit('meter')>,
 'tooth_tip_depth': 0.0015 <Unit('meter')>,
 'slot_opening': 0.0015 <Unit('meter')>,
 'tooth_width': 0.009800000000000001 <Unit('meter')>,
 'stator_outer_diameter': 0.136 <Unit('meter')>,
 'back_iron_thickness': 0.0055 <Unit('meter')>,
 'stator_internal_radius': 0.0005 <Unit('meter')>,
 'number_slots': 12 <Unit('count')>,
 'tooth_tip_angle': 1.2217304763960306 <Unit('radian')>}

In [91]:
int_per_unit_stator_params = base_unit_stator_params.copy()
for key in int_per_unit_stator_params:
    if int_per_unit_stator_params[key].dimensionality is base_unit_stator_params['stator_bore'].dimensionality:
        int_per_unit_stator_params[key] = int((base_unit_stator_params[key]/base_unit_stator_params['stator_bore']).magnitude * 2**15) 
    elif int_per_unit_stator_params[key].dimensionality is base_unit_stator_params['tooth_tip_angle'].dimensionality:
        int_per_unit_stator_params[key] = int((base_unit_stator_params[key]/np.pi).magnitude * 2**15)
    else:
        int_per_unit_stator_params[key] = int(base_unit_stator_params[key].magnitude)

In [92]:
per_unit_stator_params = base_unit_stator_params.copy()
for key in per_unit_stator_params:
    if per_unit_stator_params[key].dimensionality is base_unit_stator_params['stator_bore'].dimensionality:
        per_unit_stator_params[key] = (base_unit_stator_params[key]/base_unit_stator_params['stator_bore']).magnitude 
    elif per_unit_stator_params[key].dimensionality is base_unit_stator_params['tooth_tip_angle'].dimensionality:
        per_unit_stator_params[key] = (base_unit_stator_params[key]/np.pi).magnitude 
    else:
        per_unit_stator_params[key] = base_unit_stator_params[key].magnitude

In [94]:
base_unit_stator_params

{'slot_liner_thikness': 0.0003 <Unit('meter')>,
 'stator_bore': 0.08199999999999999 <Unit('meter')>,
 'tooth_tip_depth': 0.0015 <Unit('meter')>,
 'slot_opening': 0.0015 <Unit('meter')>,
 'tooth_width': 0.009800000000000001 <Unit('meter')>,
 'stator_outer_diameter': 0.136 <Unit('meter')>,
 'back_iron_thickness': 0.0055 <Unit('meter')>,
 'stator_internal_radius': 0.0005 <Unit('meter')>,
 'number_slots': 12 <Unit('count')>,
 'tooth_tip_angle': 1.2217304763960306 <Unit('radian')>}

In [95]:
per_unit_stator_params

{'slot_liner_thikness': 0.003658536585365854,
 'stator_bore': 1.0,
 'tooth_tip_depth': 0.01829268292682927,
 'slot_opening': 0.01829268292682927,
 'tooth_width': 0.11951219512195126,
 'stator_outer_diameter': 1.658536585365854,
 'back_iron_thickness': 0.06707317073170732,
 'stator_internal_radius': 0.006097560975609757,
 'number_slots': 12,
 'tooth_tip_angle': 0.3888888888888889}

In [80]:
import hashlib

In [81]:
hashlib.sha256(str(per_unit_stator_params).encode()).hexdigest()

'4c173a35daf2dbe3c14386650e96fee1ff1bc6c000aef996c8ce63b7c5695f44'

In [82]:
hashlib.sha256(str(base_unit_stator_params).encode()).hexdigest()

'01201a23775175778b0216b93959a114879185b8dc7fb218a011777f9b5af7bc'

In [83]:
str(base_unit_stator_params)

"{'slot_liner_thikness': <Quantity(0.0003, 'meter')>, 'stator_bore': <Quantity(0.082, 'meter')>, 'tooth_tip_depth': <Quantity(0.0015, 'meter')>, 'slot_opening': <Quantity(0.0015, 'meter')>, 'tooth_width': <Quantity(0.0098, 'meter')>, 'stator_outer_diameter': <Quantity(0.136, 'meter')>, 'back_iron_thickness': <Quantity(0.0055, 'meter')>, 'stator_internal_radius': <Quantity(0.0005, 'meter')>, 'number_slots': <Quantity(12, 'count')>, 'tooth_tip_angle': <Quantity(1.22173048, 'radian')>}"